In [4]:
from google.colab import drive
drive.mount('/content/drive')
way = "/content/drive/MyDrive/Colab_Notebooks/aaa_ml/datasets/"

Mounted at /content/drive


In [ ]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%config Completer.use_jedi = False

def get_mape(y_predict, y_true):
    return (abs(y_predict - y_true) / y_true).mean()

def process_rooms_number(x):

    if pd.isna(x):
        return 1

    if isinstance(x, int):
        return x

    if x.isdigit():
        return int(x)

    if x == 'Студия':
        return 1

    if x == 'Своб. планировка':
        return 1

    if x == '> 9':
        return 10

    return 1

# Реализуем линейную регрессию

<p>Чаще всего алгоритмы машинного обучения реализуются в виде классов с обязательными методами <code>.fit()</code>, <code>.predict()</code>. </p>

<p><code>.fit()</code> – обучить алгоритм на обучающей выборке;</p>

<p><code>.predict()</code> – сделать предсказание на тестовых данных.</p>

<p> </p>

In [ ]:
class LinearRegression:

    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001,
                 print_every=100, batch_size=None,
                 l2_coef = 0.1):

        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        self.l2_coef = l2_coef

        self.weights = None
        self.bias = None

        self.batch_size = batch_size

    def fit(self, X_train, y_train, X_val, y_val):

        self.check_regression_X_y(X_train, y_train)
        self.check_regression_X_y(X_val, y_val)

        n, m = X_train.shape

        self.weights = np.zeros((m, 1))
        self.bias = 0

        n_iter = 0
        gradient_norm = np.inf

        while n_iter < self.max_iter and gradient_norm > self.tol:

            if self.batch_size:
                random_ids = np.random.randint(0, X_train.shape[0], size=self.batch_size)
                dJdw, dJdb = self.grads(X_train[random_ids], y_train[random_ids])
            else:
                dJdw, dJdb = self.grads(X_train, y_train)

            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))

            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb

            n_iter += 1

            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)

        return self

    def predict(self, X):
        return X.dot(self.weights) + self.bias

    def grads(self, X, y):

        y_hat = self.predict(X)
        # добавим ругялризацию
        dJdw = ((y_hat - y) * X).mean(axis=0, keepdims=True).T + 2 * self.l2_coef * self.weights
        dJdb = (y_hat - y).mean()

        self.check_grads(dJdw, dJdb)

        return dJdw, dJdb

    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):

        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)

        MAPE_train = get_mape(train_preds, y_train)
        MAPE_val = get_mape(val_preds, y_val)

        print(f'{n_iter} completed. MAPE on train: {MAPE_train}, val: {MAPE_val},  grad norm: {gradient_norm}')


    def check_grads(self, dJdw, dJdb):

        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным '
                             f'числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')

        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')


    @staticmethod
    def check_regression_X_y(X, y):

        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')

        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')

        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')

        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')

        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')

        if np.any([(not isinstance(value, numbers.Real)) for value in y.flatten()]):
            raise ValueError(f'Ответы на объектах должны быть действительными числами!')


чтение ввода, и запуск фит предикта

In [ ]:
def read_input():
    n, m, k = map(int, input().split())

    x_train = np.array([input().split() for _ in range(n)]).astype(float)
    y_train = np.array([input().split() for _ in range(n)]).astype(float)
    x_test = np.array([input().split() for _ in range(k)]).astype(float)
    return x_train, y_train, x_test

def solution():

    x_train, y_train, x_test = read_input()

    model = LinearRegression(max_iter=5000, lr=1e-3, l2_coef=2.) # не меняйте гиперпараметры модели
    model.fit(x_train, y_train)

    predictions = model.predict(x_test)

    result = ' '.join(map(lambda x: str(float(x)), predictions))
    print(result)

solution()

3 4 3 1 0 0 0 0 1 0 0 0 0 1 0 1 2 3 1 0 0 0 0 1 0 0 0 0 1 0


ValueError: ignored

<h3>Тестируем модель на простой задаче</h3>

In [ ]:
X = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])
y = np.array([[1], [2], [3], [4]])
model = LinearRegression(lr=0.1)
model.fit(X, y, X, y)
model.predict(X)

100 completed. MAPE on train: 0.04256362465226591, val: 0.04256362465226591,  grad norm: 0.034121612608709384
200 completed. MAPE on train: 0.006958616076559935, val: 0.006958616076559935,  grad norm: 0.005261242720739285
300 completed. MAPE on train: 0.0016163910724505959, val: 0.0016163910724505959,  grad norm: 0.0012425153423383014


array([[1.00251522],
       [2.0021799 ],
       [3.00184458],
       [3.9962243 ]])

# Решаем задачу предсказания цены

In [ ]:
data = pd.read_csv(way + 'real_estate_novosibirsk.csv')

<p>Чистим данные:</p>

In [ ]:
data = data.drop_duplicates(subset=['item_id'], keep='last')
data = data.dropna(subset=['area'])
data['rooms_number'] = data['rooms_number'].apply(process_rooms_number).copy()
data = data[(data.price > 970000) & (data.price < 12700000)]
data = data[(data.floor < 59)]

data = data.dropna(axis=0)

In [ ]:
train, val, train_price, val_price = train_test_split(data.drop('price', axis=1), data['price'], random_state=42)

In [ ]:
test_data = pd.read_csv(way + 'test_price_estimator.csv')
test, test_price = test_data.drop('price', axis=1), test_data['price']

y_train = train_price.values.reshape(-1, 1)
y_val = val_price.values.reshape(-1, 1)

### Делаем бейзлайн

In [ ]:
get_mape(y_predict=np.median(y_train), y_true=y_val)

0.37614684131639187

### Обучаем модель

1) Начинаем с простого

In [ ]:
X_train = train[['area']].values
X_val = val[['area']].values


model = LinearRegression(lr=6e-4, max_iter=140000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.2403071091778094, val: 0.23296605233177475,  grad norm: 30178.394614846664
20000 completed. MAPE on train: 0.24564446364510098, val: 0.23812363750806964,  grad norm: 10082.519724816922
30000 completed. MAPE on train: 0.24761698849716143, val: 0.24004124125874393,  grad norm: 3368.5424721470235
40000 completed. MAPE on train: 0.24829650560283892, val: 0.24070513761386608,  grad norm: 1125.4208963981048
50000 completed. MAPE on train: 0.248525808270847, val: 0.24092876306204308,  grad norm: 376.00006665263305
60000 completed. MAPE on train: 0.24860254998529435, val: 0.24100356434937206,  grad norm: 125.6206016569753
70000 completed. MAPE on train: 0.24862819983322224, val: 0.24102856758505173,  grad norm: 41.969502030282065
80000 completed. MAPE on train: 0.24863677079040228, val: 0.24103692109822367,  grad norm: 14.021896706972033
90000 completed. MAPE on train: 0.24863963432393132, val: 0.2410397119842998,  grad norm: 4.6846776288660115
100000 complete

<p>Для того, чтобы начать ориентироваться в метрике решения задачи, очень важно построить одну или несколько простых моделей. Часто есть соблазн добавить все признаки сразу и обучить модель — мы так поступать не будем. Наоборот, мы будем постепенно добавлять признаки и следить за тем, что модель решает задачу лучше и лучше. </p>

2) Увеличиваем количество признаков

In [ ]:
X_train = train[['area', 'floors_in_house', 'floor']].values
X_val = val[['area', 'floors_in_house', 'floor']].values

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.23917853832475514, val: 0.23096363711133588,  grad norm: 8084.745826294776
20000 completed. MAPE on train: 0.23823602088558732, val: 0.2300955829491724,  grad norm: 4089.0133325021848
30000 completed. MAPE on train: 0.2377892341009792, val: 0.22969079671964643,  grad norm: 2068.095941742555
40000 completed. MAPE on train: 0.2375687951452407, val: 0.22949315208203522,  grad norm: 1045.978693748834
50000 completed. MAPE on train: 0.2374591114643267, val: 0.22939464715777072,  grad norm: 529.0235359461394
60000 completed. MAPE on train: 0.23740423107596897, val: 0.2293452285901117,  grad norm: 267.56367338793984
70000 completed. MAPE on train: 0.23737664688318014, val: 0.22932037578572795,  grad norm: 135.32539566272573
80000 completed. MAPE on train: 0.23736270325286457, val: 0.22930784019524159,  grad norm: 68.44338201590804
90000 completed. MAPE on train: 0.2373556580644064, val: 0.22930150008292005,  grad norm: 34.616536821096815
100000 completed. MAP

Делаем новые признаки
##### One hot encoding

In [ ]:
ohe_example = pd.DataFrame({'feature': ['a', 'b', 'a', 'c']})
ohe_example

,feature
0,a
1,b
2,a
3,c


In [ ]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(ohe_example)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [ ]:
ohe_house_type_transformer = OneHotEncoder(sparse=False)
train_ohe_house_type = ohe_house_type_transformer.fit_transform(train[['type_of_house']])
val_ohe_house_type = ohe_house_type_transformer.transform(val[['type_of_house']])

ohe_district_transformer = OneHotEncoder(sparse=False)
train_ohe_district = ohe_district_transformer.fit_transform(train[['district']])
val_ohe_district = ohe_district_transformer.transform(val[['district']])

X_train_extended = np.hstack([X_train, train_ohe_house_type, train_ohe_district])
X_val_extended = np.hstack([X_val, val_ohe_house_type, val_ohe_district])

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


10000 completed. MAPE on train: 0.20997591903450094, val: 0.20298104724144572,  grad norm: 22251.92057227795
20000 completed. MAPE on train: 0.20834308443293764, val: 0.20134601586236767,  grad norm: 4968.5850880222115
30000 completed. MAPE on train: 0.20818085090083624, val: 0.2011710332883239,  grad norm: 1867.267689951973
40000 completed. MAPE on train: 0.20822128598969675, val: 0.20120311755319759,  grad norm: 1059.8921670690631
50000 completed. MAPE on train: 0.2082737052118003, val: 0.2012512016311847,  grad norm: 666.5231889474117
60000 completed. MAPE on train: 0.2083124806332238, val: 0.20128753359529014,  grad norm: 425.23397161459775
70000 completed. MAPE on train: 0.20833841582017093, val: 0.20131194861039617,  grad norm: 271.7998632079062
80000 completed. MAPE on train: 0.20835532448927388, val: 0.2013279238406587,  grad norm: 173.77068314135965
90000 completed. MAPE on train: 0.20836620459022995, val: 0.20133819773727465,  grad norm: 111.1010038302235
100000 completed. MA

In [ ]:
X_train_extended.shape

(27721, 18)

(27721, 18)

In [ ]:
model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1, batch_size=8196)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

10000 completed. MAPE on train: 0.19886746858706444, val: 0.1923323268714259,  grad norm: 14302228.515894588
20000 completed. MAPE on train: 0.20005050444790093, val: 0.19327433360245502,  grad norm: 9616031.021797758
30000 completed. MAPE on train: 0.23532936704877594, val: 0.23150458614732636,  grad norm: 39214605.17606159
40000 completed. MAPE on train: 0.35231943818446987, val: 0.341860765706847,  grad norm: 45279443.57279474
50000 completed. MAPE on train: 0.20258684178872505, val: 0.19572426890063555,  grad norm: 11184165.130913833
60000 completed. MAPE on train: 0.3544427343051589, val: 0.34393992865961803,  grad norm: 51105995.41771039
70000 completed. MAPE on train: 0.27091827130057095, val: 0.2621523115866771,  grad norm: 26869664.95008358
80000 completed. MAPE on train: 0.1971472122742125, val: 0.19166367531032044,  grad norm: 23642180.81444467
90000 completed. MAPE on train: 0.20448728936906807, val: 0.19754679543880338,  grad norm: 3555186.9575620475
100000 completed. MAPE

### Задание на семинаре: попробовать улучшить метрику MAPE до 15.8% (топ-1 без ML с первой недели).

Варианты путей для улучшения:

    1) Делать новые признаки из существующих;
    2) Препроцессинг данных, целевой переменной - постпроцессинг ответов модели;
    3) Анализ ошибок модели –> генерация идей;
    4) Добавить регуляризацию;


#Логистическая регрессия

<h3>Задание на семинаре: реализуем логистическую регрессию</h3>

<p>Мы получаем оптимальные веса алгоритма градиентным спуском:</p>

<p style="text-align:center"><br />
<br />
<span class="math-tex">\(\begin{bmatrix} w_{1}^{t+1}\\  ...\\ w_{m}^{t+1}\\  \end{bmatrix} = \begin{bmatrix} w_{1}^{t}\\  ...\\ w_{m}^{t}\\  \end{bmatrix} - \alpha \cdot  \begin{bmatrix} \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{1}^{(i)}\\  ...\\ \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{m}^{(i)}\\  \end{bmatrix}\)</span></p>

<p style="text-align:center"><span class="math-tex">\(b^{t+1} = b^{t} - \alpha \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})\)</span></p>

<p style="text-align:center">&nbsp;</p>

<p>&nbsp;</p>

In [34]:
from sklearn.metrics import accuracy_score


class MyLogisticRegression:

    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=200, l1_coef = 0.1):

        '''
        max_iter: максимальное количеств
        lr: коэффициент изменения градиента
        tol: ограничение изменения градиента для остановки работы
        print_every: через сколько писать промужточные результаты
        '''

        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        self.l1_coef = l1_coef

        self.weights = None
        self.bias = None

    def fit(self, X_train, y_train, X_val, y_val):

        '''
        Обучение модели.

        X_train – матрица объектов для обучения
        y_train – ответы на объектах для обучения

        X_val – матрица объектов для валидации
        y_val – ответы на объектах для валидации
        '''

        #self.check_binary_clf_X_y(X_train, y_train)
        #self.check_binary_clf_X_y(X_val, y_val)

        n, m = X_train.shape

        y_train = y_train.reshape(-1, 1)
        y_val = y_val.reshape(-1, 1)

        self.weights = np.zeros((m, 1))
        self.bias = 0

        n_iter = 0
        gradient_norm = np.inf

        while n_iter < self.max_iter and gradient_norm > self.tol:

            dJdw, dJdb = self.grads(X_train, y_train)
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))

            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb

            n_iter += 1

            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)

        return self

    def predict(self, X):

        '''
        Метод возвращает предсказанную метку класса на объектах X
        '''

        return self.predict_proba(X) > 0.5


    def predict_proba(self, X):

        '''
        Метод возвращает вероятность класса 1 на объектах X
        '''
        z = X.dot(self.weights) + self.bias
        return self.sigmoid(z)


    def grads(self, x, y):
        '''
        Рассчёт градиентов
        '''
        y_hat = self.predict_proba(x)
        error = y_hat - y
        #print(np.sign(self.weights).shape)
        #print((error * x).mean(axis=0, keepdims=True).T.shape)
        dJdw = np.mean(x * error, axis=0, keepdims=True).T
        dJdw += self.l1_coef * (np.sign(self.weights).reshape(-1, 1))
        dJdb = error.mean()

        return dJdw, dJdb

    @staticmethod
    def sigmoid(x):
        '''
        Сигмоида от x
        '''
        x_clipped = np.clip(x, -200, 200)
        return 1 / (1 + np.exp(-x_clipped))

    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):

        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)

        train_acc = accuracy_score(train_preds, y_train)
        val_acc = accuracy_score(val_preds, y_val)

        print(f'{n_iter} completed. accuracy_score on train: {train_acc}, val: {val_acc}, grad_norm: {gradient_norm}')

    def check_grads(self, dJdw, dJdb):

        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным'
                             f' числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')

        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')

    @staticmethod
    def check_binary_clf_X_y(X, y):

        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')

        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')

        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')

        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')

        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')


        if sorted(np.unique([1, 0, 0])) != [0, 1]:
            raise ValueError(f'Ответы на объектах должны быть только 0 или 1, а у нас np.unique(y) = {np.unique(y)}')


# Домашнее задание


<p>Воспользуемся реализованной моделью логистической регрессии, чтобы решить задачу определения пола пользователя Авито.</p>

<p><a href="https://stepik.org/media/attachments/lesson/527992/binary_clf_data.csv" rel="noopener noreferrer nofollow">Данные</a> даны в сыром виде &ndash; айтемы и их категории, которые выкладывали покупатели на Авито. Целевая переменная: <em>gender.</em></p>

<p>Вам необходимо разбить данные на train, val. Перед загрузкой файла с ответом убедитесь, что точность (<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html" rel="noopener noreferrer nofollow">accuracy</a>)&nbsp;на валидации не менее 0.7.</p>

<p>&nbsp;</p>

<p><strong>План действий</strong></p>

<p>Сначала нужно преобразовать категории с помощью one-hot encoding. Далее необходимо агрегировать категории, в которых пользователи выкладывали объявления, чтобы получить вектор признаков для каждого объекта. В результате у каждого пользователя будет вектор признаков, содержащий количество айтемов, выложенных в каждой из возможных категорий.</p>

<ul>
	<li>Убедитесь, что для каждого пользователя в выборке есть только один объект, каждый признак означает количество айтемов, выложенное этим пользователем в категории;</li>
	<li>Убедитесь, что после one-hot энкодинга каждая категория соответствует признаку,&nbsp;<strong>одинаковому в train, val и test.</strong></li>
</ul>

<p>Попробуйте варианты отбора признаков. Для борьбы с переобучением на редких категориях используйте регуляризацию. В качестве&nbsp;ответа загрузите файл с предсказанием пола для пользователей:</p>

<p style="text-align:center">&nbsp;</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th style="text-align:center">user_id</th>
			<th style="text-align:center">gender</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td style="text-align:center">15424171</td>
			<td style="text-align:center">male</td>
		</tr>
		<tr>
			<td style="text-align:center">15454025</td>
			<td style="text-align:center">female</td>
		</tr>
	</tbody>
</table>

<p style="text-align:center">&nbsp;</p>

<p>Такой файл можно сформировать с помощью&nbsp;<code>test_predictions.to_csv(&#39;test_predictions.csv&#39;, index=False)</code>.</p>

<p>После того, как получилось обучить модель, ответьте на вопрос: какие из категорий вносят наибольший вклад в вероятность класса &quot;мужчина&quot; и класса &quot;женщина&quot;?</p>

<p>Например, если вы закодировали &quot;мужчина&quot; как 1, большие положительные веса при признаках будут означать большой вклад в вероятность класса 1, большие по модулю отрицательные веса будут вносить наибольший вклад в вероятность класса 0. Согласуется ли полученный результат с вашим жизненным опытом?</p>


In [1]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [5]:
df_sales = pd.read_csv(way + "binary_clf_data.csv")
df_sales = df_sales[["gender", "user_id", "category_id", "subcategory_id", "param1"]]
df_sales['gender'] = df_sales['gender'].replace({'male': 0, 'female': 1})
df_sales.sample(5)

,gender,user_id,category_id,subcategory_id,param1
5885,1,154240052,49,10,NaN
2827,0,154212824,17,26,Телевизоры и проекторы
547,0,154197003,12,34,"Шины, диски и колёса"
4102,1,154221497,28,47,Велосипеды и самокаты
2652,1,154212744,28,57,Парфюмерия


In [6]:
df_sales2 = pd.read_csv(way + "dataset_527992_9.txt")
df_sales2 = df_sales2[["user_id", "category_id", "subcategory_id", "param1"]]
df_sales2.sample(2)

,user_id,category_id,subcategory_id,param1
1692,154229530,17,41,iPhone
876,154203485,17,26,Телевизоры и проекторы


In [7]:
df_train = df_sales.copy()
df_train.shape

(8923, 5)

In [15]:
def df_modification(df, ohe_category_transformer, ohe_subcategory_transformer, test_flag = False):

  df_category = ohe_category_transformer.transform(df[['category_id']])
  df_subcategory = ohe_subcategory_transformer.transform(df[['param1']])

  category_feature_names = ohe_category_transformer.get_feature_names_out(['category_id'])
  subcategory_feature_names = ohe_subcategory_transformer.get_feature_names_out(['param1'])

  df_category = pd.DataFrame(df_category, columns=category_feature_names)
  df_subcategory = pd.DataFrame(df_subcategory, columns=subcategory_feature_names)

  df_category = pd.DataFrame(df_category, columns=category_feature_names)
  df_subcategory = pd.DataFrame(df_subcategory, columns=subcategory_feature_names)

  if test_flag:
    df_dropped = df.drop(['category_id', 'subcategory_id', 'param1'], axis=1)
    df_final = pd.concat([df_dropped.reset_index(drop=True), df_category,  df_subcategory], axis=1)
    df_final = df_final.groupby("user_id").sum()
    return df_final.reset_index()

  df_dropped = df.drop(['category_id', 'subcategory_id', "gender", 'param1'], axis=1)
  genders = df[["user_id", "gender"]].drop_duplicates()

  df_final = pd.concat([df_dropped.reset_index(drop=True), df_category, df_subcategory], axis=1)
  df_final = df_final.groupby("user_id").sum()
  df_final = df_final.merge(genders, on='user_id', how='left')

  return df_final

In [16]:
ohe_category_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_category_transformer.fit(df_train[["category_id"]])

ohe_subcategory_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_subcategory_transformer.fit(df_train[["param1"]])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [17]:
df_train_mod = df_modification(df_train, ohe_category_transformer, ohe_subcategory_transformer)

In [18]:
df_train_mod.sample(5)

,user_id,category_id_1,category_id_7,category_id_12,category_id_17,category_id_28,category_id_39,category_id_49,category_id_55,category_id_250002,...,param1_Шотландская,"param1_Шоу, мюзикл",param1_Шпиц,param1_Экипировка,param1_Электронные книги,"param1_Этикетки, бутылки, пробки",param1_Ювелирные изделия,param1_Юриспруденция,param1_nan,gender
826,154223829,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
716,154217682,0.0,2.0,0.0,1.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1032,154231516,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1517,154249041,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1697,154255001,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [19]:
train, val, train_gender, val_gender = train_test_split(df_train_mod.drop('gender', axis=1), df_train_mod['gender'], random_state=21)
print(train.shape, val.shape, train_gender.shape, val_gender.shape)

(1437, 281) (479, 281) (1437,) (479,)


In [ ]:
#все включаем
train, train_gender = df_train_mod.drop('gender', axis=1), df_train_mod['gender']

In [38]:
model = MyLogisticRegression(l1_coef = 0.1, max_iter=10000, lr = 0.004)

model.fit(np.array(train.values), np.array(train_gender).reshape(-1, 1))

val_predictions = model.predict(val)
accuracy_score(val_gender, val_predictions)

<ipython-input-34-d8e46814fdab>:88: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


0.3945720250521921

In [20]:
from sklearn.linear_model import LogisticRegression

C_value = 10000

model = LogisticRegression(C = C_value, solver='sag', max_iter=1000000)

model.fit(train.drop(["user_id"], axis = 1), train_gender)

val_predictions = model.predict(val.drop(["user_id"], axis = 1))
accuracy_score(val_gender, val_predictions)

0.6993736951983298

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Создаем экземпляр модели случайного леса
model1 = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучаем модель на тренировочных данных
model1.fit(train.drop(["user_id"], axis=1), train_gender)

# Делаем предсказания на валидационном наборе
val_predictions1 = model1.predict(val.drop(["user_id"], axis=1))

# Вычисляем точность
accuracy1 = accuracy_score(val_gender, val_predictions1)
print(f"Accuracy: {accuracy1}")


Accuracy: 0.7077244258872651


In [28]:
val_predictions

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,

In [22]:
df_test_mod = df_modification(df_sales2, ohe_category_transformer, ohe_subcategory_transformer, test_flag = True)

In [23]:
test_predictions = model1.predict(df_test_mod.drop(["user_id"], axis = 1))

In [30]:
ans_df = pd.concat([df_test_mod["user_id"], pd.Series(test_predictions).replace({0: 'male', 1: 'female'})],
                   axis = 1).rename({0: "gender"}, axis = 1)
ans_df.to_csv('test_predictions.csv', index=False)

In [29]:
ans_df

,user_id,gender
0,154189609,male
1,154189728,female
2,154189879,female
3,154189983,female
4,154190268,female
...,...,...
634,154271834,male
635,154272200,female
636,154272848,female
637,154272955,male


In [ ]:
train_gender.sum() / train_gender.shape[0]

0.5845511482254697